In [1]:
import tensorflow as tf
import os
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("MINIST_data",one_hot=True)

batch_size = 50

n_batch = mnist.train.num_examples // batch_size

#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev = 0.1)
    return tf.Variable(initial)

#初始化偏置
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#卷积层
def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding = 'SAME')

#池化层
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1],strides = [1,2,2,1],padding='SAME')
with tf.device('/device:GPU:0'):
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    #placeholder
    x = tf.placeholder(tf.float32,[None,784])
    y = tf.placeholder(tf.float32,[None,10])
    #reshape for image
    x_image = tf.reshape(x,[-1,28,28,1])

    #初始化第一个卷积层
    # 卷积大小（5x5）通道（1）输出层(32)
    w_conv1 = weight_variable([5,5,1,32])
    b_conv1 = bias_variable([32])

    #卷积及池化操作
    h_conv1 = tf.nn.relu(conv2d(x_image,w_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    #第二个卷积层

    w_conv2 = weight_variable([5,5,32,64])
    b_conv2 = bias_variable([64])

    #卷积及池化操作
    h_conv2 = tf.nn.relu(conv2d(h_pool1,w_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    #两层处理后 结果为 7x7大小

    #初始化第一个全连接层的权值
    w_fc1 = weight_variable([7*7*64,1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

    #初始化第二个全连接层
    w_fc2 = weight_variable([1024,10])
    b_fc2 = bias_variable([10])

    prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,w_fc2) + b_fc2)

    #交叉熵代价函数
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y,logits = prediction))
    #优化器
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(10):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict = {x:batch_xs,y:batch_ys,keep_prob:0.7})
        
        acc = sess.run(accuracy,feed_dict = {x:mnist.test.images,y:mnist.test.labels,keep_prob:0.7})
        
        print("Iter " + str(epoch) + ",Testing accuracy " + str(acc))

Extracting MINIST_data\train-images-idx3-ubyte.gz
Extracting MINIST_data\train-labels-idx1-ubyte.gz
Extracting MINIST_data\t10k-images-idx3-ubyte.gz
Extracting MINIST_data\t10k-labels-idx1-ubyte.gz


ResourceExhaustedError: OOM when allocating tensor with shape[10000,32,14,14] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node MaxPool_4 (defined at <ipython-input-4-07cc2f4e6d61>:23) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[node Mean_5 (defined at <ipython-input-4-07cc2f4e6d61>:74) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'MaxPool_4', defined at:
  File "C:\ProgramData\Anaconda3\envs\tf\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\envs\tf\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()

In [15]:
config = tf.ConfigProto()

In [ ]:
config.gpu_options.